In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.core.display import display

# display(HTML("<style>div.output pre { white-space: pre;}</style>"))

# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

# Width of the display in characters. If set to None and pandas will correctly auto-detect the width.
pd.set_option('display.width', None)

In [ ]:
home = Path(os.getenv("HOME"))
LOGS_PATH = home / "batch_jobs"
LOGS_PATH.mkdir(parents=True, exist_ok=True)

parsed_file_pattern = "jobs_*.csv"


In [ ]:
# jobs = pd.read_csv(jobs_csv)
import dask.dataframe as dd
ddf = dd.read_csv(LOGS_PATH / "**" / parsed_file_pattern)
jobs = ddf.compute()

jobs['created'] = pd.to_datetime(jobs.created)
jobs['started'] = pd.to_datetime(jobs.started)
jobs['stopped'] = pd.to_datetime(jobs.stopped)
jobs['elapsed'] = jobs.stopped - jobs.created
jobs['runtime'] = jobs.stopped - jobs.started
jobs['spinup'] = jobs.started - jobs.created
jobs.info()


# Successful Jobs

These are only for SUCCEEDED jobs

In [ ]:
success_jobs = jobs.query('status == "SUCCEEDED"')
np.unique(success_jobs.status)

job = success_jobs.iloc[-1].copy()
job.created, job.started, job.stopped, job.spinup, job.runtime, job.elapsed

## Job Spinup
AWS does not charge for the spinup time on AWS-Batch.
This is the delay between job-submission (`created_at`) and the
start of the job (`started_at`).  This can differ for cold-start
vs warm-start jobs because the docker image needs to be transferred
into a new EC2-node for a batch compute environment.


In [ ]:
display(success_jobs.runtime.sum())

In [ ]:
display(success_jobs.spinup.describe())

## Job Runtime
These are job runtime, after spinup.

In [ ]:
display(success_jobs.runtime.sum())

In [ ]:
display(success_jobs.runtime.describe())

## Elapsed Time
These are total job time, from job submission to completion.

These values are not very useful, since they do not account for the parallel execution of all the batch jobs.  The AWS-Batch compute environment will specify a limit to the available cores for concurrent jobs.  The job-definition could require N-cores per job.

In [ ]:
display(success_jobs.elapsed.sum())

In [ ]:
# Assume there are 500 parallel batch jobs, then elapsed time is
display(success_jobs.elapsed.sum() / 500)

In [ ]:
display(success_jobs.elapsed.describe())